## **サンプル: ホテル予約のためのマルチAIエージェント**

現代の忙しい社会では、出張の計画は単にフライトやホテルを予約するだけでは済みません。調整や効率性が求められるため、それを達成するのは容易ではありません。ここでマルチAIエージェントが登場し、私たちの旅行管理の方法を革新します。

想像してみてください。あなたの旅行のあらゆる側面を正確かつ簡単に処理するために協力する、知能を持ったエージェントのチームが手元にいるとしたら。私たちの先進的なAI技術を活用し、予約サービスや旅程の手配に特化したエージェントを作成しました。これにより、スムーズでストレスのない旅行体験を実現します。

これは基本的なシナリオです。出張を計画する際には、航空券情報やホテル情報を得るためにビジネストラベルエージェントに相談する必要があります。AIエージェントを活用することで、予約サービス用のエージェントや旅程手配用のエージェントを構築し、協力して知能のレベルを向上させることができます。


# Azure AI Agent Service を初期化し、**.env** から設定情報を取得する

### **.env**

.env ファイルを作成する

**.env** には、Azure AI Agent Service の接続文字列、AOAI が使用するモデル、対応する Google API 検索サービスの API、ENDPOINT などが含まれます。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Azure AI Agent Service モデル展開名"

[**NOTE**] モデルには、100,000 のレート制限（1 分あたりのトークン数）と 600 のレート制限（1 分あたりのリクエスト数）が必要です。

  モデルは Azure AI Foundry - Model and Endpoint で取得できます。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Azure AI Agent Service プロジェクト接続文字列"

  プロジェクト接続文字列は、AI Foundry ポータル画面のプロジェクト概要で取得できます。

- **SERPAPI_SEARCH_API_KEY** = "SERPAPI 検索 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "SERPAPI 検索 ENDPOINT"

Azure AI Agent Service のモデル展開名とプロジェクト接続文字列を取得するには、Azure AI Agent Service を作成する必要があります。以下の [テンプレート](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) を使用して直接作成することを推奨します。（***注意:*** Azure AI Agent Service は現在、限定されたリージョンで設定されています。[このリンク](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) を参照してリージョンを設定することを推奨します）

Agent は SERPAPI にアクセスする必要があります。[このリンク](https://serpapi.com/searches) を使用して登録することを推奨します。登録後、ユニークな API KEY と ENDPOINT を取得できます。


# Azure にログインする

Azure にログインする必要があります。VScode のターミナルを開き、`az login` コマンドを実行してください。


# セットアップ

このノートブックを実行するには、以下のライブラリをインストールする必要があります。必要なライブラリと対応する pip インストールコマンドのリストは以下の通りです:

azure-identity: Azure認証用。  
requests: HTTPリクエストを行うため。  
semantic-kernel: セマンティックカーネルフレームワーク用（これがカスタムまたは特定のライブラリである場合、特定のソースやリポジトリからインストールする必要があるかもしれません）。  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# 説明:
import asyncio: asyncioモジュールをインポートします。このモジュールはPythonで非同期プログラミングをサポートしており、asyncとawait構文を使用して並行処理のコードを書くことができます。
from typing import Annotated: typingモジュールからAnnotated型をインポートします。Annotatedは型ヒントにメタデータを追加するために使用され、検証、ドキュメント作成、ツールの利用などさまざまな目的に役立ちます。


In [ ]:
import asyncio,os
from typing import Annotated

# 説明:
`from dotenv import load_dotenv` と `load_dotenv()` を使用することで、設定情報や機密情報（例えば、APIキーやデータベースURL）を `.env` ファイルで簡単に管理できます。これにより、ソースコードからそれらを分離し、アプリケーションをより安全かつ設定しやすくすることができます。


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 説明:

インポート文: from azure.identity.aio import DefaultAzureCredential  
この文は、azure.identity.aioモジュールからDefaultAzureCredentialクラスをインポートしています。モジュール名に含まれるaioは、このモジュールが非同期操作向けに設計されていることを示しています。

DefaultAzureCredentialの目的:  
DefaultAzureCredentialクラスは、Azure SDK for Pythonの一部です。このクラスは、Azureサービスに認証するためのデフォルトの方法を提供します。環境変数、マネージドID、Azure CLI資格情報など、特定の順序で複数の方法を試みて認証を行います。

非同期操作:  
aioモジュールは、DefaultAzureCredentialクラスが非同期操作をサポートしていることを示しています。これにより、asyncioを使用して非ブロッキングの認証リクエストを実行することが可能です。


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# 説明:
semantic_kernelパッケージからさまざまなモジュールやクラスをインポートしています。以下は各インポートの概要です:

semantic_kernel.agentsのAgentGroupChat: このクラスは、AIエージェントのグループチャットに関連する機能を扱います。semantic_kernel.agents.azure_aiのAzureAIAgentとAzureAIAgentSettings:

AzureAIAgent: Azure AIサービスを利用するAIエージェントを作成および管理するためのクラスです。

AzureAIAgentSettings: AzureAIAgentの設定を構成するためのクラスです。semantic_kernel.agents.strategies.termination.termination_strategyのTerminationStrategy:

このクラスは、特定の条件下でAIエージェントの実行を終了するための戦略を定義します。semantic_kernel.contents.chat_message_contentのChatMessageContent:

このクラスは、チャットメッセージの内容を扱うために使用されます。
semantic_kernel.contents.utils.author_roleのAuthorRole:

このクラスは、チャットメッセージの文脈で著者の異なる役割を定義します。

semantic_kernel.functions.kernel_function_decoratorのkernel_function: このデコレーターは、semantic kernelフレームワーク内で実行可能なカーネル関数を定義するために使用されます。
これらのインポートは、ホテル予約などの活動を含む可能性があるグループチャット環境でAIエージェントを作成および管理するための必要なコンポーネントを設定します。


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# 説明:
次に、azure.ai.projects.models モジュールから CodeInterpreterTool クラスをインポートします。

CodeInterpreterTool: このクラスは Azure AI SDK の一部であり、AI プロジェクトの文脈でコードを解釈し実行するために使用されます。コードスニペットの実行、コードの分析、または AI ワークフロー内でのコード実行の統合といった機能を提供します。  
このインポートにより、プロジェクト内で CodeInterpreterTool を利用するための必要なコンポーネントが設定されます。これは、コードを動的に解釈・実行するタスクに役立つ可能性があります。


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# 説明:
ApprovalTerminationStrategyクラスは、AIエージェントの動作を終了させるための特定の戦略を提供します。この戦略では、エージェントの対話履歴の最後のメッセージに「saved」という単語が含まれている場合に終了します。この方法は、何かが「保存」されたという確認を受け取った時点でエージェントのタスクが完了したとみなされるシナリオで役立つ可能性があります。対話方法を定義します。予約プランが保存された後、保存された信号を受け取ることで停止することができます。


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# 説明:

このコード行は、create() メソッドを呼び出すことで、デフォルトまたは事前定義された設定を持つ AzureAIAgentSettings オブジェクトを初期化します。この設定オブジェクト (ai_agent_settings) は、その後 AzureAIAgent インスタンスを設定および管理するために使用できます。


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# 説明:
requestsライブラリをインポートすることで、Pythonコード内で簡単にHTTPリクエストを送信し、ウェブサービスとやり取りすることができます。


In [ ]:
import requests

# 説明:
これは、SERP（検索エンジン結果ページ）APIサービスにアクセスするためのAPIキーを格納する変数です。APIキーは、アカウントに関連付けられたリクエストを認証するために使用される一意の識別子です。

'GOOGLE_SEARCH_API_KEY': これはプレースホルダー文字列です。'GOOGLE_SEARCH_API_KEY' を実際のSERP APIキーに置き換える必要があります。

目的: この行の目的は、APIキーを変数に格納し、SERP APIサービスへのリクエストを認証するために使用できるようにすることです。APIキーは、サービスにアクセスして検索を実行するために必要です。

SERP APIキーの取得方法: SERP APIキーを取得するには、以下の一般的な手順に従ってください（使用する特定のSERP APIサービスによって手順が異なる場合があります）:

1. SERP APIサービスを選ぶ: SerpAPI、Google Custom Search JSON APIなど、いくつかのSERP APIサービスがあります。自分のニーズに最も合ったものを選んでください。

2. アカウントに登録する:

選んだSERP APIサービスのウェブサイト（例: https://www.serpapi.com）にアクセスし、アカウントを作成してください。基本的な情報を提供し、メールアドレスを確認する必要がある場合があります。

3. APIキーを作成する:

登録後、アカウントにログインし、APIセクションまたはダッシュボードに移動します。新しいAPIキーを作成または生成するオプションを探してください。

4. APIキーをコピーする:

APIキーが生成されたら、それをコピーしてください。このキーは、SERP APIサービスへのリクエストを認証するために使用されます。

5. プレースホルダーを置き換える:

.envファイル内のプレースホルダーを、コピーしたAPIキーに置き換えてください。


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# 説明:
BookingPluginクラスは、Serpapi.comのGoogle Search APIを使用してホテルやフライトを予約するためのメソッドを提供します。このクラスは必要なパラメータを構築し、APIリクエストを送信し、レスポンスを処理して関連する予約情報を返します。APIキー（SERPAPI_SEARCH_API_KEY）とエンドポイント（SERPAPI_SEARCH_ENDPOINT）は、Google Search APIへの認証とリクエスト送信に使用されます。


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# 説明:
SavePluginクラスは、Azure AIサービスを使用して旅行プランを保存するためのsaving_planメソッドを提供します。このクラスはAzureの認証情報を設定し、AIエージェントを作成し、ユーザー入力を処理して旅行プランの内容を生成・保存し、ファイルの保存やクリーンアップ操作を行います。メソッドは正常に完了すると「Saved」を返します。


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# 説明:
このコードは、Azure AIエージェントを設定して、ユーザーの入力に基づいてフライトやホテルの予約を行い、旅行プランを保存する機能を提供します。Azureの認証情報を使用してエージェントを作成・設定し、グループチャットを通じてユーザーの入力を処理します。また、タスク完了後に適切なクリーンアップを行うことを保証します。エージェントは、それぞれのタスクを実行するために特定のプラグイン（BookingPluginとSavePlugin）を使用します。


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は一切の責任を負いません。
